In [1]:
# 输入数据的参数
_INPUT1='{"name":"input1","type":0,"uri":"tmp_35196d820e8643b6a20a45dfd3d48005"}'
_INPUT4='{"name":"input4","type":0,"uri":"tmp_04bab065ee8044d291c5c1a66593516c"}'
_INPUT2='{"name":"input2","type":0,"uri":"tmp_7b3070b338b74dd4a4aa623192757469"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_778cef1c9c0f4698844a0a092020266d"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_60e209e947a441cd987ba904458a5fb3"},{"name":"output2","type":0,"uri":"tmp_ca98f25ba95c438eb692abc2c9892db3"},{"name":"output3","type":0,"uri":"tmp_b1ed163f4cf8491dbe195aefcf5eee8c"},{"name":"output4","type":0,"uri":"tmp_a1eae39ea2824958b509d6687c0318e3"}]'

# 自定义参数


In [2]:
!pip install logitboost
!pip install xgboost
!pip install tensorflow

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [3]:
# 引入使用库
import pandas as pd
# from ustciscrLab_A import get_score
from ustciscrBDL_B import get_score
import wfio
import random
import numpy as np
rng = np.random.RandomState(31337) 

from sklearn.preprocessing import StandardScaler  #用于标准化
from sklearn.preprocessing import MinMaxScaler  #用于归一化

import time # 用于计时
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error    # 精度模块
from sklearn.model_selection import KFold   # 交叉验证模块
from sklearn.ensemble import RandomForestClassifier    # 随机森林模块
from sklearn.tree import DecisionTreeClassifier   # 决策树模块
from sklearn.ensemble import GradientBoostingClassifier # GradientBoosting模块
from xgboost.sklearn import XGBClassifier    # XGboost模块
from sklearn.ensemble import BaggingClassifier    # Bagging模块
from sklearn.neighbors import KNeighborsClassifier   # KNn模块
from sklearn.ensemble import ExtraTreesClassifier   # ExtraTrees模块
from logitboost import LogitBoost    # LogitBoost模块
from sklearn.ensemble import AdaBoostClassifier   # AdaBoost模块
from sklearn.linear_model import LogisticRegression  # 对数几率

In [4]:
# 读取测试集数据
def get_test_data():
    test_data = wfio.read_dataframe(_INPUT3)
    # test_data = wfio.read_dataframe(_INPUT4)
    # 删除第一行中文 
    # test_data = test_data.drop(0,axis=0,inplace=False)
    return test_data

In [5]:
# 读取训练集数据
def get_train_data():
    train_data_1 = wfio.read_dataframe(_INPUT1)
    train_data_2 = wfio.read_dataframe(_INPUT2)
    train_data_1 = train_data_1.rename(columns = {'xyzl':'hyzl', 'xyml':'hyml', 'xy':'hy', 'xydl':'hydl',
                                                  'zczby':'zczb', 'jdxz':'xzjd', 'djkyrq':'djrq',
                                                  'frsjh':'fdbrxmp', 'cwryxm':'bsrxm', 'cwrysjh':'bsrxmmp'})
    # 两个数据拼一起
    train_data = pd.concat([train_data_2, train_data_1], sort=False)
    #删除第一行中文
    #train_data = train_data.drop(0,axis=0,inplace=False)
    train_data = train_data.sort_values('yc', ascending = False)
    return train_data

In [6]:
# 计算理论入库
def cal_llrk(X):    
    # X_new = np.copy(X)
    for i in range(0,96,8):
        jxse = X[:,i+1] * X[:,i+2]  # 进项税额
        llrk = np.zeros_like(jxse)  # 理论入库
        for j in range(X.shape[0]):
            if X[j,i+2] == 0.03:    # 如果进项税率为0.03
                llrk[j] = X[j,i+5]    # 理论入库为开票税额
            else:               # 如果进项税额不为0.03
                llrk[j] = max(X[j,i+5] - jxse[j], 0) # 理论入库为开票税额-进项税额（结果为负时取0）
        X = np.column_stack((X,llrk))
        # X = np.column_stack((X,llrk-X[:,i+7]))
        # X[:,i+5] = llrk
    return X

In [7]:
def cal_jxse(X):    # 计算进项税额
    for i in range(0,96,8):
        X = np.column_stack((X,X[:,i+1]*X[:,i+2]))
    return X

In [8]:
def get_djrq(djrq):  #计算登记日期
    djrq_num = np.zeros_like(djrq)
    for i in range(djrq.shape[0]):
        rq_ls = djrq[i].split(sep= '-') 
        djrq_num[i] = int(rq_ls[0])-2 + 0.25*(int(rq_ls[1])//4)
    return djrq_num

In [9]:
def cal_chsj_kpjj(X0, X): #计算存活时间和开票季节数
    djrq_num = get_djrq(X0[:,99])
    chsj = np.zeros_like(djrq_num)
    kpjj = np.zeros_like(djrq_num)
    for j in range(X.shape[0]): #遍历
        current = djrq_num[j]
        for i in range(0,96,8):
            if X[j,i+1] != 0:       #如果当季有开票
                kpjj[j] += 1        #季节加1
                current = 2017 + i/8 * 0.25     #当前季节
                chsj[j] = current - djrq_num[j]     #更新存活时间
    X_new = np.column_stack((X,chsj,kpjj))
    return X_new

In [10]:
#获取数据
def get_data():
    #训练集
    train_data = get_train_data()
    hyml = np.array(pd.get_dummies(train_data['hyml']) )
    X0 = np.array(train_data)  #所有特征
    # X = X0[:,0:96]  #数字特征
    X = np.column_stack( (X0[:,0:96],X0[:, [98, -2]] ) )
    y = np.array(train_data['yc'])  #标签

    for i in range(0,X.shape[0]):
        for j in range(0,X.shape[1]):
            try:
                X[i,j] = float(X[i,j])
            except:
                X[i,j] = 0
    X = X.astype(np.float)
    y = y.astype(np.float)

    #测试集
    test_data = get_test_data()
    Xtest0 = np.array(get_test_data())  #所有特征
    # Xtest = Xtest0[:,0:96]  #数字特征
    Xtest = np.column_stack( (Xtest0[:, 0:96], Xtest0[:, [98, -2]] ) )
    for i in range(0, Xtest.shape[0]):
        for j in range(0, Xtest.shape[1]):
            try:
                Xtest[i,j] = float(Xtest[i,j])
            except:
                Xtest[i,j] = 0
    Xtest = Xtest.astype(np.float)
    
    return train_data, test_data, X0, Xtest0, X, y, Xtest

In [11]:
# 数据处理
def data_solve(train_data, test_data, X0, Xtest0, X, Xtest, y):
    X1 = cal_llrk(X)  #计算理论入库
    X2 = cal_jxse(X1)  #计算进项税额
    X3 = cal_chsj_kpjj(X0, X2) #计算开票季节、存活时间
    Xtest1 = cal_llrk(Xtest)
    Xtest2 = cal_jxse(Xtest1)
    Xtest3 = cal_chsj_kpjj(Xtest0, Xtest2) 

    #行业门类
    from sklearn.preprocessing import LabelEncoder
    hyml_co = pd.concat([train_data['hyml'], test_data['hyml']],sort=False)
    hyml_np = LabelEncoder().fit_transform(hyml_co.values)
    #行业大类
    hydl_co = pd.concat([train_data['hydl'], test_data['hydl']],sort=False)
    hydl_np = LabelEncoder().fit_transform(hydl_co.values)
    #乡镇街道
    xzjd_co = pd.concat([train_data['xzjd'], test_data['xzjd']],sort=False)
    xzjd_np = LabelEncoder().fit_transform(xzjd_co.values)
    X4 = np.column_stack((X3, hyml_np[0: X.shape[0]], hydl_np[0: X.shape[0]], xzjd_np[0: X.shape[0]]) )
    Xtest4 = np.column_stack((Xtest3, hyml_np[X.shape[0]: ], hydl_np[X.shape[0]: ], xzjd_np[X.shape[0]: ]) )

    #标准化
    stdsc = StandardScaler()
    X_std = stdsc.fit_transform(X)
    Xtest_std = stdsc.transform(Xtest)
    X1_std = stdsc.fit_transform(X1)
    Xtest1_std = stdsc.transform(Xtest1)
    X2_std = stdsc.fit_transform(X2)
    Xtest2_std = stdsc.transform(Xtest2)
    X3_std = stdsc.fit_transform(X3)
    Xtest3_std = stdsc.transform(Xtest3)
    X4_std = stdsc.fit_transform(X4)
    Xtest4_std = stdsc.transform(Xtest4)

    #归一化
    mms = MinMaxScaler()
    X_norm = mms.fit_transform(X)
    Xtest_norm = mms.transform(Xtest)
    X1_norm = mms.fit_transform(X1)
    Xtest1_norm = mms.transform(Xtest1)
    X2_norm = mms.fit_transform(X2)
    Xtest2_norm = mms.transform(Xtest2)
    X3_norm = mms.fit_transform(X3)
    Xtest3_norm = mms.transform(Xtest3)
    X4_norm = stdsc.fit_transform(X4)
    Xtest4_norm = stdsc.transform(Xtest4)
    
    #特征名称
    characters = list(train_data.columns[:96])
    characters += ['cyrs' , 'zczb']

    characterjd = characters[0:96:8]
    def ll(x):
        x = x.replace('fphdsl','llrk')
        return x
    llrk_s = map(ll, characterjd)
    characters += llrk_s
    def kp(x):
        x = x.replace('fphdsl','kpse')
        return x
    kpse_s = map(kp, characterjd)
    characters += kpse_s
    characters += ['chsj', 'kpjj', 'hyml' , 'hydl', 'xzjd']   
    
    return X4_std, Xtest4_std, characters

In [12]:
def char_selct(X4, Xtest4, characters): #特征筛选
    X5 = pd.DataFrame(X4, columns = characters)
    Xtest5 = pd.DataFrame(Xtest4, columns = characters)

    col = list(range(6,96,8) ) + list(range(2,96,8) ) + list(range(4,96,8) )
    X6 = X5.drop(['zczb', 'cyrs'], axis=1)
    Xtest6 = Xtest5.drop(Xtest5.columns[[96,97]], axis=1)

    col = list(range(6,96,8) ) + list(range(2,96,8) ) + list(range(4,96,8) )
    X7 = X6.drop(X5.columns[col], axis=1)
    Xtest7 = Xtest6.drop(Xtest5.columns[col], axis=1)

    X8 = X7.drop(['hydl'], axis=1)
    Xtest8 = Xtest7.drop(['hydl'], axis=1)
    
    return X5,Xtest5, X6, Xtest6, X7, Xtest7, X8, Xtest8

In [13]:
#打分函数
def create_verify_train_data(train_data, test_data, predictions):
    #读取训练集的'zjnsrsbh'字段
    train_data = test_data
    prob = {"zjnsrsbh": train_data['zjnsrsbh'],}
    train_data = pd.DataFrame(prob)
    train_data['Probability'] = predictions
    return train_data

In [14]:
#模型训练
# model = SVC(kernel='linear', gamma='scale', probability=True, C=100).fit(X1[0:3500,:], y[0:3500])
# C为正则化系数的倒数
# model = LogisticRegression(solver= 'lbfgs', max_iter=2000, C=10).fit(X1[0:3000,:], y[0:3000])  #对数几率
# model = LogisticRegression(solver= 'lbfgs', max_iter=2000, C=10 ** 7).fit(X4_std[0:7250,:], y[0:7250])  #对数几率(7250)
# model = RandomForestClassifier(n_estimators=125).fit(X4[0:7250,:], y[0:7250])  #随机森林
# model = LogitBoost().fit(X4[0:2000,:], y[0:2000])  #logitboost
# model = DecisionTreeClassifier().fit(X4[0:7300,:], y[0:7300])  #决策树
# model = KNeighborsClassifier(n_neighbors=2,p=2,metric='minkowski').fit(X4[0:7250,:], y[0:7250])
# model = BaggingClassifier().fit(X4_std[0:7250,:], y[0:7250])
# model = ExtraTreesClassifier(n_estimators=100).fit(X4_std[0:7250,:], y[0:7250])
# model = AdaBoostClassifier().fit(X4_std[0:7250,:], y[0:7250])
# model = GradientBoostingClassifier(n_estimators=125).fit(X4_std[0:7000,:], y[0:7000])
# model = XGBClassifier(learning_rate=0.1,n_estimatores=150,min_child_weight = 1).fit(X4_std[0:8000,:], y[0:8000])

#获取数据    
train_data, test_data, X0, Xtest0, X, y, Xtest = get_data() 

#开始计时    
get_score.post_user_id('5') 
#数据预处理
X4_std, Xtest4_std, characters = data_solve(train_data, test_data, X0, Xtest0, X, Xtest, y) 
#特征筛选
X5,Xtest5, X6, Xtest6, X7, Xtest7, X8, Xtest8 = char_selct(X4_std, Xtest4_std, characters)
#模型训练（对数几率）    
model = LogisticRegression(solver= 'lbfgs', max_iter=2000, C=10**7).fit(X5.values[200:7350,:], y[200:7350])  
#数据预测
predictions = model.predict_proba(Xtest5.values)[:,1] #在测试集上进行预测

#打分
get_score.post_verify_data(create_verify_train_data(train_data, test_data, predictions)) 

User-id:5
程序开始运行时间为：2019-12-22 13:41:11.468373
程序结束运行时间为：2019-12-22 13:41:13.276576
程序运行时间（去除打分耗时）为：1.808203
程序的准确率为：95.75373406
最终得分为：96.17836065
